In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

data = pd.read_csv("Maternal Health Risk Data Set.csv")

label_encoder = LabelEncoder()
data['RiskLevel'] = label_encoder.fit_transform(data['RiskLevel'])

X = data.drop('RiskLevel', axis=1)
y = data['RiskLevel']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

model = Sequential([
    Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=4)


c:\Users\sench\miniforge3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4635 - loss: 1.0292 - val_accuracy: 0.5443 - val_loss: 0.8228
Epoch 2/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6161 - loss: 0.7621 - val_accuracy: 0.6131 - val_loss: 0.7262
Epoch 3/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6818 - loss: 0.6881 - val_accuracy: 0.6590 - val_loss: 0.6900
Epoch 4/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7204 - loss: 0.6674 - val_accuracy: 0.6656 - val_loss: 0.6724
Epoch 5/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7227 - loss: 0.6387 - val_accuracy: 0.6787 - val_loss: 0.6666
Epoch 6/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7635 - loss: 0.5827 - val_accuracy: 0.7016 - val_loss: 0.6345
Epoch 7/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7432 - loss: 0.5886 - val_accuracy: 0.6951 - val_loss: 0.6362
Epoch 8/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7113 - loss: 0.6167 - val_accu

In [13]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

y_pred = np.argmax(model.predict(X_test), axis=-1)

print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.82      0.82        76
           1       0.75      0.73      0.74       117
           2       0.68      0.71      0.69       112

    accuracy                           0.74       305
   macro avg       0.75      0.75      0.75       305
weighted avg       0.74      0.74      0.74       305

Confusion Matrix:
 [[62  4 10]
 [ 5 85 27]
 [ 9 24 79]]


In [14]:
from keras.layers import Input
from keras.models import Model

def build_autoencoder(input_dim, encoding_dim):
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(encoding_dim, activation='relu')(input_layer)
    decoded = Dense(input_dim, activation='sigmoid')(encoded)
    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder

autoencoder1, encoder1 = build_autoencoder(X_train.shape[1], 64)
autoencoder1.fit(X_train, X_train, epochs=100, batch_size=4, shuffle=True)

X_train_encoded = encoder1.predict(X_train)
autoencoder2, encoder2 = build_autoencoder(64, 32)
autoencoder2.fit(X_train_encoded, X_train_encoded, epochs=100, batch_size=4, shuffle=True)

X_train_encoded = encoder2.predict(X_train_encoded)
autoencoder3, encoder3 = build_autoencoder(32, 16)
autoencoder3.fit(X_train_encoded, X_train_encoded, epochs=100, batch_size=4, shuffle=True)

model.layers[0].set_weights(autoencoder1.get_weights()[:2])
model.layers[1].set_weights(autoencoder2.get_weights()[:2])
model.layers[2].set_weights(autoencoder3.get_weights()[:2])

model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=4)


Epoch 1/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.1109
Epoch 2/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7193
Epoch 3/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5821
Epoch 4/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6048
Epoch 5/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5453
Epoch 6/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5479
Epoch 7/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5448
Epoch 8/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5169
Epoch 9/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5304
Epoch 10/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5704
Epoch 11/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5273
Epoch 12/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5286
Epoch 13/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5281
Epoch 14/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5488
Epoch 15/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 0

In [15]:
y_pred_pretrained = np.argmax(model.predict(X_test), axis=-1)

print("Classification Report (Pretrained):\n", classification_report(y_test, y_pred_pretrained))
print("Confusion Matrix (Pretrained):\n", confusion_matrix(y_test, y_pred_pretrained))


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Classification Report (Pretrained):
               precision    recall  f1-score   support

           0       0.73      0.89      0.80        76
           1       0.68      0.81      0.74       117
           2       0.82      0.53      0.64       112

    accuracy                           0.73       305
   macro avg       0.74      0.74      0.73       305
weighted avg       0.74      0.73      0.72       305

Confusion Matrix (Pretrained):
 [[68  6  2]
 [11 95 11]
 [14 39 59]]
